In [7]:
import pandas as pd
import numpy as np
import string
import re # regular expressions
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from joblib import Parallel, delayed 
import joblib 

import utils

In [5]:
df = pd.read_csv("./datas/temizlenmemis_tweets.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

,username,tweet
0,netflixturkiye,@valenciaessek Mutlu yıllar!
1,netflixturkiye,Yeni sezon şimdi yayında.
2,netflixturkiye,Geceye yeni yıldan bir beklenti bırak.
3,netflixturkiye,"Özür dilerim ben de spoiler verdim, 20 puanla ..."
4,netflixturkiye,2022 yılı hatalar testi:\n\nSpoiler verdim (20...


In [8]:
def clean(tweet):
    """
    This function converts tweets into a simpler form by removing meaningless words, punctuation marks, and numbers.

    Parameters:
    tweet (str): The tweet to be cleaned.

    Returns:
    row (str): The cleaned tweet.
    """
    satirlar = tweet.replace("\n", " ")  # Replace "\n" with " " (space) in the dataset.
    satirlar = re.sub("[0-9]+", "", satirlar) # Remove numbers from the data.
    satirlar = [t for t in satirlar.split()if t not in utils.stop_words] # Remove meaningless words from the data.
    
    return " ".join(satirlar).lower().translate(str.maketrans("", "", string.punctuation)) # Clean punctuation marks from the data.

In [9]:
df["clean"] = df.apply(lambda row: clean(row["tweet"]), axis = 1)
df

,username,tweet,clean
0,netflixturkiye,@valenciaessek Mutlu yıllar!,valenciaessek mutlu yıllar
1,netflixturkiye,Yeni sezon şimdi yayında.,yeni sezon şimdi yayında
2,netflixturkiye,Geceye yeni yıldan bir beklenti bırak.,geceye yeni yıldan bir beklenti bırak
3,netflixturkiye,"Özür dilerim ben de spoiler verdim, 20 puanla ...",özür dilerim ben de spoiler verdim puanla kapa...
4,netflixturkiye,2022 yılı hatalar testi:\n\nSpoiler verdim (20...,yılı hatalar testi spoiler verdim puan önerdi...
...,...,...,...
11460,RasimOzan_K,Bu şerefsizliğe alet olarak bu haysiyetsizce y...,bu şerefsizliğe alet olarak bu haysiyetsizce y...
11461,RasimOzan_K,Aleni yalanlarla bana bu şerefsizliği yapan fi...,aleni yalanlarla bana bu şerefsizliği yapan fi...
11462,RasimOzan_K,Bu yalan açıklamaları yapan şahsa ve dolandırı...,bu yalan açıklamaları yapan şahsa ve dolandırı...
11463,RasimOzan_K,Ben asla böyle bir dilekçe vermedim.Hakkımda b...,ben asla böyle bir dilekçe vermedimhakkımda ba...


In [10]:
# users in dataset
users = list(df["username"].unique())
users

['netflixturkiye', 'webtekno', 'acunilicali', 'RasimOzan_K']

In [11]:
def etiketle(row, users):
    
    """
    Performs Label Encoding on the dataset.
    
    Parameters:
    row (str): The row to be Label Encoded.
    users (list): The list of users present in the dataset to be referenced during Label Encoding.
    
    Returns:
    (int)
    """

    if row["username"] == users[0]:
        return 0 
    elif row["username"] == users[1]:
        return 1
    elif row["username"] == users[2]:
        return 2
    return 3

In [12]:
# Performing Label Encoding on the dataset
df["labels"] = df.apply(lambda row:  etiketle(row, users), axis=1)
df

,username,tweet,clean,labels
0,netflixturkiye,@valenciaessek Mutlu yıllar!,valenciaessek mutlu yıllar,0
1,netflixturkiye,Yeni sezon şimdi yayında.,yeni sezon şimdi yayında,0
2,netflixturkiye,Geceye yeni yıldan bir beklenti bırak.,geceye yeni yıldan bir beklenti bırak,0
3,netflixturkiye,"Özür dilerim ben de spoiler verdim, 20 puanla ...",özür dilerim ben de spoiler verdim puanla kapa...,0
4,netflixturkiye,2022 yılı hatalar testi:\n\nSpoiler verdim (20...,yılı hatalar testi spoiler verdim puan önerdi...,0
...,...,...,...,...
11460,RasimOzan_K,Bu şerefsizliğe alet olarak bu haysiyetsizce y...,bu şerefsizliğe alet olarak bu haysiyetsizce y...,3
11461,RasimOzan_K,Aleni yalanlarla bana bu şerefsizliği yapan fi...,aleni yalanlarla bana bu şerefsizliği yapan fi...,3
11462,RasimOzan_K,Bu yalan açıklamaları yapan şahsa ve dolandırı...,bu yalan açıklamaları yapan şahsa ve dolandırı...,3
11463,RasimOzan_K,Ben asla böyle bir dilekçe vermedim.Hakkımda b...,ben asla böyle bir dilekçe vermedimhakkımda ba...,3


In [13]:
X = df.clean.to_numpy() # Converting cleaned tweets to a numpy array
y = df.labels.to_numpy() # Converting labels of tweets to a numpy array

# Splitting the dataset into 80% training and 20% test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converting tweets into vectors using the TF-IDF method
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### What is Multinomial Naive Bayes?
Multinomial Naive Bayes (MultinomialNB) is a probabilistic classification algorithm based on Bayes' theorem and certain assumptions. This algorithm is particularly suitable for text classification problems based on word counts. Naive Bayes classifiers are simple yet powerful and fast algorithms.

In [14]:
# Training the Multinomial Naive Bayes model
model_NB = MultinomialNB()
model_NB.fit(X_train, y_train)

# Model's acc on the dataset
print("NB train accuracy:", model_NB.score(X_train, y_train))
print("NB test accuracy:", model_NB.score(X_test, y_test))

NB train accuracy: 0.9615133013519407
NB test accuracy: 0.8634976013955517


### What is Decision Tree Classifier?
Decision Tree Classifier is a classifier used for classification problems. Decision trees classify the examples in a dataset by splitting them into branches based on their features.

In [15]:
# Training the Decision Tree Classifier model
model_DT = DecisionTreeClassifier()
model_DT.fit(X_train, y_train)

# Model's acc on the dataset
print("DT train accuracy:", model_DT.score(X_train, y_train))
print("DT test accuracy:", model_DT.score(X_test, y_test))

DT train accuracy: 0.9996729175752289
DT test accuracy: 0.7056258177060619


## Test

In [16]:
tweet = clean("İzlediğin son filmi söyle, nasıl hissettiğini tahmin edeyim.")
tweet_vec = vectorizer.transform([tweet])
# Netflix: 0
# Webtekno: 1
# Acun: 2
# ROK: 3
print("NB:", model_NB.predict(tweet_vec))
print("DT:", model_DT.predict(tweet_vec))

result_NB = model_NB.predict_proba(tweet_vec)
result_DT = model_DT.predict_proba(tweet_vec)
print(
    f"Netflix: \tNB: {round(100*result_NB[0][0],2)} \tDT: {round(100*result_DT[0][0],2)}\n"
    f"Webtekno: \tNB: {round(100*result_NB[0][1],2)} \tDT: {round(100*result_DT[0][1],2)}\n"
    f"Acun Ilicali: \tNB: {round(100*result_NB[0][2],2)} \tDT: {round(100*result_DT[0][2],2)}\n"
    f"Rasim Ozan Kutahyali: \tNB: {round(100*result_NB[0][3],2)} \tDT{round(100*result_DT[0][3],2)}"
    )

NB: [0]
DT: [1]
Netflix: 	NB: 77.82 	DT: 0.0
Webtekno: 	NB: 17.24 	DT: 100.0
Acun Ilicali: 	NB: 0.27 	DT: 0.0
Rasim Ozan Kutahyali: 	NB: 4.66 	DT0.0


In [17]:
def tweet_analyse(tweet):
    tweet = clean(tweet)
    tweet_vec = vectorizer.transform([tweet])
    # Netflix: 0
    # Webtekno: 1
    # Acun: 2
    # ROK: 3
    print("NB:", model_NB.predict(tweet_vec))
    print("DT:", model_DT.predict(tweet_vec))
    
    result_NB = model_NB.predict_proba(tweet_vec)
    result_DT = model_DT.predict_proba(tweet_vec)
    print(
        f"Netflix: \tNB: {round(100*result_NB[0][0],2)} \tDT: {round(100*result_DT[0][0],2)}\n"
        f"Webtekno: \tNB: {round(100*result_NB[0][1],2)} \tDT: {round(100*result_DT[0][1],2)}\n"
        f"Acun Ilicali: \tNB: {round(100*result_NB[0][2],2)} \tDT: {round(100*result_DT[0][2],2)}\n"
        f"Rasim Ozan Kutahyali: \tNB: {round(100*result_NB[0][3],2)} \tDT{round(100*result_DT[0][3],2)}"
        )

In [18]:
tweet_analyse("İzlediğin son filmi söyle, nasıl hissettiğini tahmin edeyim.")

NB: [0]
DT: [1]
Netflix: 	NB: 77.82 	DT: 0.0
Webtekno: 	NB: 17.24 	DT: 100.0
Acun Ilicali: 	NB: 0.27 	DT: 0.0
Rasim Ozan Kutahyali: 	NB: 4.66 	DT0.0


### save the models

In [19]:
joblib.dump(vectorizer, "./models/vectorizer.pkl")

['./models/vectorizer.pkl']

In [20]:
joblib.dump(model_NB, './models/model_NB.pkl')
joblib.dump(model_DT, './models/model_DT.pkl')

['./models/model_DT.pkl']